In [1]:
import time
import vlc
import os

# creating the mediaplayer object and setting the file to be run on.
# defining default values for volume and scale.

print(os.getcwd())

media_player = vlc.MediaPlayer()
media = vlc.Media("video-for-executing.mkv") 
media_player.set_media(media)
media_player.audio_set_volume(80) 
media_player.audio_set_mute(False) 
media_player.video_set_scale(0.6) 

c:\Users\akhee\Documents\Projects\NITK\sem4\Seminar 290\Hand_Gesture_Recognition-main\Hand_Gesture_Recognition-main\Hand_Gesture_Recognition-main\Code


In [2]:
#importing required packages
import tensorflow as tensorflow
from tensorflow import keras


from keras.models import Sequential,load_model

In [3]:
#importing required packages
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.layers import Dense,Dropout,Flatten

import numpy as np
import matplotlib.pyplot as plt

import cv2
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.utils import np_utils



bg = None

#resize and save captured frames 
from PIL import Image
def resizeImage(imageName):
    basewidth = 100
    img = Image.open(imageName)
    wpercent = (basewidth/float(img.size[0]))
    hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((basewidth,hsize), Image.ANTIALIAS)
    img.save(imageName)

In [4]:
def run_avg(image, aWeight):
    global bg
    # initialize the background
    if bg is None:
        bg = image.copy().astype("float")
        return
    # compute weighted average, accumulate it and update the background
    cv2.accumulateWeighted(image, bg, aWeight)

In [5]:
# figuring out the shape of the hand or highlighted contour part of hand from the input video

def segment(image, threshold=25):
    global bg
    # find the absolute difference between background and current frame
    diff = cv2.absdiff(bg.astype("uint8"), image)

    # threshold the diff image so that we get the foreground
    thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)[1]

    # get the contours in the thresholded image
    (cnts, _) = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # return None, if no contours detected
    if len(cnts) == 0:
        return
    else:
        # based on contour area, get the maximum contour which is the hand
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)

In [6]:
import matplotlib.pyplot as plt 

def getPredictedClass():
    # Predict
    image = cv2.imread('Temp.png') 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_image = cv2.resize(gray_image, (128,128))
    
    gray_image=gray_image.reshape(1,128,128,1)
    prediction = model.predict(gray_image)
    #calculate the probablity as a ratio between gesture of max probablity and total probablity of all gestures
    
    p=(np.amax(prediction) / (prediction[0][0] + prediction[0][1] + prediction[0][2] + prediction[0][3] + prediction[0][4] + prediction[0][5] + prediction[0][6] + prediction[0][7] + prediction[0][8] + prediction[0][9] ))
    return np.argmax(prediction), (p)
    #assigning action to gesture
    
def showStatistics(predictedClass, confidence):

    textImage = np.zeros((300,512,3), np.uint8)
    className = "lmao"
    if predictedClass == 0:
        className = "Down"
        var="Volume decrease"
        print(className)
        media_player.audio_set_volume(media_player.audio_get_volume()-30)
    elif predictedClass == 1:
        className = "Palm"
        var="Unmute"
        print(className)
        media_player.audio_set_mute(False) 
    elif predictedClass == 2:
        className = "L"
        var="Rewind"
        print(className)
        media_player.set_time(media_player.get_time()-10000)
    elif predictedClass == 3:
        className = "Fist" 
        var="Mute"
        print(className)
        media_player.audio_set_mute(True)
    elif predictedClass == 4:
        className = "Fist_Moved"
        var="Maximize"
        print(className)
        media_player.video_set_scale(1)
    elif predictedClass == 5:
        className = "Thumb" 
        var="Play"
        print(className)
        media_player.set_pause(0)
    elif predictedClass == 6:
        className = "Index"
        var="Volume increase"
        print(className)
        media_player.audio_set_volume(media_player.audio_get_volume()+30)
    elif predictedClass == 7:
        className = "Ok"
        var="Restore Down"
        print(className)
        media_player.video_set_scale(0.6) 
    elif predictedClass == 8:
        className = "Palm_Moved"
        var="Pause"
        print(className)
        media_player.set_pause(1)
    elif predictedClass == 9:
        className = "C"
        var="Forward"
        print(className)
        media_player.set_time(media_player.get_time()+10000)
    else : 
        className = "none"
        print(className)
    cv2.putText(textImage,"Predicted Class : " + className, (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(textImage,"Action : " + var, (30, 100), cv2.FONT_HERSHEY_SIMPLEX, 1,(255, 255, 255),2)
    cv2.putText(textImage,"Probablity : " + str(confidence) + '%', (30, 170), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.imshow("Statistics", textImage)
    cv2.waitKey(0)

In [7]:
def main():
    # initialize weight for running average
    aWeight = 0.5
    flag=0
    t=0
    j=0
    m=[]
    for i in range(0,10) :
        m.append(0)
    
    # get the reference to the webcam
    camera = cv2.VideoCapture(0)

    # region of interest (ROI) coordinates
    top, right, bottom, left = 10, 350, 225, 590

    # initialize num of frames
    num_frames = 0
    start_recording = False
    
    # keep looping, until interrupted
    while(True):
            
        # get the current frame
        (grabbed, frame) = camera.read()

        # resize the frame
        frame = imutils.resize(frame, width = 700)

        # flip the frame so that it is not the mirror view
        frame = cv2.flip(frame, 1)

        # clone the frame
        clone = frame.copy()

        # get the height and width of the frame
        (height, width) = frame.shape[:2]

        # get the ROI
        roi = frame[top:bottom, right:left]

        # convert the roi to grayscale and blur it
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)
        gray = cv2.dilate(gray, None, iterations=4)
        # to get the background, keep looking till a threshold is reached
        # so that our running average model gets calibrated
        if num_frames < 30:
            run_avg(gray, aWeight)
            
        else:
            # segment the hand region
            hand = segment(gray)

            # check whether hand region is segmented
            if hand is not None:
                # if yes, unpack the thresholded image and
                # segmented region
                (thresholded, segmented) = hand

                # draw the segmented region and display the frame
                cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))
                if start_recording:
                    cv2.imwrite('Temp.png', thresholded)
                    resizeImage('Temp.png')
                    predictedClass, confidence = getPredictedClass()
                    #print(predictedClass)
                    if flag>=1 :
                        m[predictedClass]+=1
                        flag+=1
                cv2.imshow("Thesholded", thresholded)  
                #for the same gesture 50 frames are taken and are predicted . Out of 50 predictions , the gesture having maximum probability is considered as final gesture 
                if flag==50 :
                    print('in flag')
                    t=0
                    for i in range(0,10) :
                        print(i," th = ",m[i])
                        if m[i]>t :
                            t=m[i]
                            j=i;
                    print("final",t)
                    print("---",j)
                    showStatistics(j, t*2)
                    flag=0
                    
                    
        # draw the segmented hand
        cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)

        # increment the number of frames
        num_frames += 1

        # display the frame with segmented hand
        cv2.imshow("Video Feed", clone)

        # observe the keypress by the user
        keypress = cv2.waitKey(1) & 0xFF

        # if the user pressed "q", then stop looping
        if keypress == ord("q"):
            break
        
        if keypress == ord("s"):
            start_recording = True
            time.sleep(5) 
            media_player.play() 
        
        if keypress == ord(" "):
            print("Gesture prediction started")
            flag=1
            for i in range(0,10) :
                m[i]=0

In [8]:
import imutils
import tensorflow

# Load Saved Model

model = tensorflow.keras.models.load_model("handgestures.h5")

main()

Gesture prediction started
in flag
0  th =  0
1  th =  0
2  th =  0
3  th =  1
4  th =  0
5  th =  0
6  th =  0
7  th =  0
8  th =  48
9  th =  0
final 48
--- 8
Palm_Moved
